In [3]:
import os
import numpy as np
import pandas as pd
import pickle5 as pkl
import plotly.express as px
from keypoint_util import KeypointSelector


data_folder = '../TargetObjectHeuristic/heuristic_test/'
data497 = '1645721497'

with open(os.path.join(data_folder, data497, 'labeled_actions_new.pkl'), 'rb') as f:
    label_actions = pkl.load(f)
trajectory_file = os.path.join(data_folder, data497, 'obj_pose_trajectory_interpolated_median.h5')
trajectory_dataset = pd.read_hdf(trajectory_file)
all_objs = ['tap', 'pitcher', 'cup', 'teabag1', 'teabag2']

selector = KeypointSelector(1, 2)

In [ ]:
instance = label_actions[4]
moving_obj = instance['moving_obj']
moving_obj_traj = []
target_obj_traj = {}
df_lst = []
for idx in range(instance['start'], instance['end']):
    row = trajectory_dataset[moving_obj].iloc[idx]
    moving_obj_traj.append([row.x, row.y, row.z])
model_df = pd.DataFrame(np.array(moving_obj_traj).reshape(-1, 3))
model_df['obj'] = moving_obj
df_lst.append(model_df)

# fill target object trajectory dict
for other_obj in all_objs:
    if other_obj!=moving_obj:
        target_obj_traj[other_obj] =  []
        for idx in range(instance['start'], instance['end']):
            row = trajectory_dataset[other_obj].iloc[idx]
            target_obj_traj[other_obj].append([row.x, row.y, row.z])
        model_df = pd.DataFrame(np.array(target_obj_traj[other_obj]).reshape(-1, 3))
        model_df['obj'] = other_obj
        df_lst.append(model_df)
        
obj_graph = pd.concat(df_lst)
keypoints = selector.search(moving_obj_traj)

In [ ]:
obj_graph = pd.concat(df_lst)
obj_graph.columns = ['x', 'y', 'z', 'obj']
obj_graph = obj_graph.iloc[keypoints]
fig = px.scatter_3d(obj_graph,x='x',y='y',z='z',color='obj')
fig = px.scatter_3d(obj_graph,x='x',y='y',z='z',color='obj')
fig.update_traces(marker=dict(size=3))
fig.show()